In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import math
import itertools
from scipy.optimize import curve_fit, minimize
import h5py
mpl.rcParams['text.usetex'] = True
%matplotlib inline
#import random #MI: millor fes servir np.random.normal
import matplotlib.patches as patches
from matplotlib.patches import Polygon
import pandas as pd

This download may take some time however **you can skip it** if you don't want to use LaTeX in your plots. Keep in mind that the functions provided to do the plots uses it. You will need to create a custom plot. But you know how to do it :)

In [ ]:
!apt-get install -y texlive texlive-fonts-recommended texlive-fonts-extra texlive-latex-extra
!sudo apt install texlive-fonts-extra cm-super
!sudo apt install dvipng -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
texlive is already the newest version (2021.20220204-1).
texlive-fonts-extra is already the newest version (2021.20220204-1).
texlive-fonts-recommended is already the newest version (2021.20220204-1).
texlive-latex-extra is already the newest version (2021.20220204-1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
cm-super is already the newest version (0.3.4-17).
texlive-fonts-extra is already the newest version (2021.20220204-1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
dvipng is already the newest version (1.15-1.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


#Importing the data set

In [GitHub](https://github.com/assumpg/TAE-School-LQCD/tree/main) we have two directories with diferent nodes in their spatial dimension 16x128 and 24x128. In each directory there are several files with the correlator data from pions, protons, xi, etc.

The code provided to read the data is done for the directory 16x128 so we recomend you use either the datafile below or one with a single particle.

```
px0py0pz0_pi_Nsrc224_Ncfg2001_16x128_um0p0840_sm0p0743_P.dat
```

*   px0py0pz0: means that the data are projected to zero moment
*   pi: pion data
*   Nsc224: number of sources taken randomly from the 2001 configurations
*   Ncfg: number rof configurations
*   16x128: means that there are 16 nodes for each spatial dimension and 128 in the temporal
*   um0p0840: mass of u and d
*   sm0p0743: mass of s
*   P: type of sink "P" means local sink (point-like), "S" means smeared sink (gaussian-type) both with smeared source


Then we need to process the data and give it the wanted structure meaning that we will build the correlator matrix $C_i(t)$.

In [ ]:
url='https://raw.githubusercontent.com/assumpg/TAE-School-LQCD/main/16x128/px0py0pz0_pi_Nsrc224_Ncfg2001_16x128_um0p0840_sm0p0743_P.dat'

#Step 1: Read the .dat file, skipping the first row (header)
df = pd.read_csv(url, delim_whitespace=True, skiprows=1, header=None)

# Step 2: Extract only the second column, because we only need the numbers in the second column
second_column = df[1]

# Step 3: Reshape the data into a DataFrame with 2001 rows and 128 columns
reshaped_df = second_column.values.reshape((2001, 128))

# Step 4: Keep only the first 60 columns
reshaped_df = reshaped_df[:, :60]

# Step 5: Convert the reshaped array into a DataFrame
final_df = pd.DataFrame(reshaped_df)

In [ ]:
#Check that the data is correct comparing with the url
final_df

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,0.007575,0.005935,0.005268,0.004681,0.004183,0.003762,0.003402,0.003089,0.002817,0.002574,...,0.002375,0.002583,0.002821,0.003095,0.003407,0.003765,0.004183,0.004678,0.005261,0.005925
1,0.007739,0.006103,0.005438,0.004848,0.004350,0.003926,0.003562,0.003250,0.002980,0.002742,...,0.002571,0.002786,0.003026,0.003298,0.003608,0.003959,0.004369,0.004860,0.005439,0.006097
2,0.007878,0.006192,0.005518,0.004924,0.004413,0.003977,0.003604,0.003277,0.002992,0.002745,...,0.002496,0.002721,0.002974,0.003259,0.003588,0.003969,0.004407,0.004919,0.005509,0.006182
3,0.007865,0.006177,0.005514,0.004929,0.004429,0.004004,0.003638,0.003316,0.003030,0.002775,...,0.002530,0.002757,0.003010,0.003291,0.003612,0.003984,0.004418,0.004920,0.005508,0.006173
4,0.007480,0.005792,0.005134,0.004557,0.004068,0.003653,0.003290,0.002973,0.002698,0.002455,...,0.002233,0.002447,0.002687,0.002961,0.003274,0.003637,0.004057,0.004549,0.005128,0.005794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1996,0.007472,0.005822,0.005165,0.004590,0.004104,0.003690,0.003334,0.003024,0.002759,0.002530,...,0.002353,0.002560,0.002790,0.003052,0.003353,0.003702,0.004108,0.004588,0.005159,0.005821
1997,0.007193,0.005582,0.004938,0.004380,0.003906,0.003498,0.003142,0.002842,0.002588,0.002372,...,0.002293,0.002477,0.002685,0.002926,0.003217,0.003563,0.003962,0.004425,0.004978,0.005608
1998,0.007755,0.006063,0.005397,0.004814,0.004320,0.003898,0.003542,0.003235,0.002961,0.002712,...,0.002516,0.002732,0.002978,0.003253,0.003571,0.003938,0.004360,0.004855,0.005434,0.006088
1999,0.008057,0.006387,0.005714,0.005123,0.004626,0.004220,0.003875,0.003567,0.003294,0.003047,...,0.002734,0.002959,0.003210,0.003495,0.003815,0.004187,0.004609,0.005106,0.005706,0.006385


# Processing the data with the bootsrap and jacknife methods

There are two methods to reduce bias from a data set, the bootstrap and the jackknife. We can compute the effective mass and the correlator with both methods obtaining minor differences between them.

In [ ]:
# Defining the constants
nsc = 2001        #Number of configurations
nt = 60          #Time intervals
nboot = nsc       #N_b

From the data frame we read the values $C_i(t)$ as:
- Each column is a time ($t_j$)
- Each row is a configuration (we have $N$ configurations)

In [ ]:
# Convert dataframe to numpy array
# This is the C_i(t) matrix size (nsc x nt)

blck = final_df.values

## Bootstrap method

From an original sample we have read $N$ configurations of the correlator $C_i(t)$.

With the bootstrap method we create $N_b$ bootstrap samples $C_b(t)$, where each one is obtained from a random selection from the original sample $N$ points (with repetitions allowed)

$$ C_b(t)=\frac{1}{N} \sum_{i=1}^{N} C_{\text{rand}(i)b}(t) $$

*Keep in mind that we want to get a random value of C for each time* $t$ *and each bootstrap sample* $b$*. The easiest way of doing it is creating a matrix* ($N \times N_b$) *with random numbers and use each element to get the* $C_{\text{rand}(i)b}(t)$.

To compute the effective mass for each bootstrap sample and its main value we use

$$ E_b=\frac{1}{t_J}\log\frac{C_b(t)}{C_b(t+t_J)} $$

with $t_J=1$ and

$$ \bar{E}(t)=\frac{1}{N_b}\sum_{b=1}^{N_b}E_b(t) $$

With this we get the effective mass for each time, and in order to compute the corresponding error we use

$$ \sigma(t)=\sqrt{\frac{N}{N_b(N-1)}\sum_{b=1}^{N_b}(E_b(t)-\bar{E}(t))^2} $$

In [ ]:
# BOOTSTRAP METHOD

# Introduce code here #

#Save mean and sigma to use later

## Jacknife

Considering an original sample containing $N$ configurations, the jackknife samples are constructed by taking the average of the variable $x$ without including the $n$th sample

$$ x_n^J =\frac{1}{N-1}\sum_{m\neq n}x_m =\frac{N}{N-1}\bar{x}-\frac{1}{N-1}x_n $$

Then the mean value of the function $f$ and estimation of its uncertainty is evaluated as:

$$ f(X)\simeq \bar{f}_x^J=\frac{1}{N}\sum_{n=1}^N f(x_n^J) $$
$$ \sigma_{f(X)}^2 \simeq \frac{N-1}{N}\sum_{n=1}^N [f(n_n^J)-\bar{f}_x^J]^2 $$

*An easy way of doing so is computing the mean of* $C_i(t) → \bar{C}(t)$ *and create a similar matrix to* $C_b(t)$ *by:*
$$C_{jack}(t)=\frac{N\bar{C}(t)-C_i(t)}{N-1}$$
*Notice that this is what you have above in terms of* $x$. *You can proceed as in the bootstrap method but with this new* $C_{jk}(t)$ *instead of* $C_b(t)$. *Take in consideration the dimensions. Here you are not using* $N_b$.

In [ ]:
# JACKNIFE METHOD

# Save the mean and sigma

## Correlator

We compute the mean and sigma correlator with both the bootstrap and the jacknife method.

* The mean of the correlator for each method in the mean of $C_b(t)$ at each time.

* The mean of the correlator for each method in the mean of $C_{jack}(t)$ at each time.

The variance is computed as usual.

### Plot function

Here we provide a function for the plots. You can use it or not. If you do install the LaTeX package.

In [ ]:
# Plot data
def plot_data(x_data, y_data_list, y_err_list, plot_title, xlabel, ylabel, xlim, ylim, scale, y_leg_list, legend, fit_line=None, fill_between=None):
    # Configure plot settings
    plt.rc('text', usetex=True)
    plt.rc('font', family='serif', size='12')

    fig, ax = plt.subplots(figsize=(8, 6))
    plt.subplots_adjust(left=0.08, bottom=0.08, right=0.98, top=0.95, wspace=0.21, hspace=0.2)

    ax.set_title(plot_title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.set_yscale(scale)
    ax.minorticks_on()
    ax.tick_params(which='both', direction='in')
    ax.yaxis.set_ticks_position('both')
    ax.xaxis.set_ticks_position('both')

    # Plot each dataset with error bars
    for i, (y_data, y_err, y_leg) in enumerate(zip(y_data_list, y_err_list, y_leg_list)):
        ax.errorbar(x_data, y_data, yerr=y_err, fmt='o', markersize=3, capsize=1, elinewidth=0.7, label=y_leg)

    if fit_line is not None:
        ax.plot(fit_line[0], fit_line[1], 'r-', label=f'fit: {fit_line[2]}')

    if fill_between is not None:
        ax.fill_between(fill_between[0], fill_between[1], fill_between[2], color='#ffcccc')

    if legend:
        ax.legend()

    # Display the plot
    plt.show()


Here you should plot the effective mass, the variance and the correlator as functions of time for both methods.

A plot example using the function provided:


```
plot_data(
    x_data=np.arange(x_size),
    y_data_list=[value1, value1],
    y_err_list=[sigma1, sigma2],
    plot_title='Title',
    xlabel=r'$x \,\mathrm{(l.u.)}$',
    ylabel=r'$\mathrm{y} \,\mathrm{(l.u.)}$',
    xlim=[x0, x1],
    ylim=[y0, y1],
    scale='linear',
    y_leg_list=['Leg1', 'Leg2'],
    legend=True
)
```



## Comparing some data

The same that we have did, can be done comparing different data. A way of proceeding is setting the bootstrap and jacknife methods as functions and apply them to diferent urls.

[Smeared pion](https://raw.githubusercontent.com/assumpg/TAE-School-LQCD/main/16x128/px0py0pz0_pi_Nsrc224_Ncfg2001_16x128_um0p0840_sm0p0743_S.dat), [proton](https://https://raw.githubusercontent.com/assumpg/TAE-School-LQCD/main/16x128/px0py0pz0_prot_Nsrc224_Ncfg2001_16x128_um0p0840_sm0p0743_P.dat), [xi](https://raw.githubusercontent.com/assumpg/TAE-School-LQCD/main/16x128/px0py0pz0_xi_Nsrc224_Ncfg2001_16x128_um0p0840_sm0p0743_P.dat)

# Fitting the data


## Linear and exponential fits

With the effective mass plotted we could mke some fits to obtain it.

By looting at the plot we set the time interval in which we will make the fit.

We compute two fits:

$$ f_l(t)=k \hspace{2cm} f_e(t)=ae^{-bt}+c $$

The linear form assumes that we are in the plateaux region, with no excited state contribution. The exponential one takes into account, additionally, one contribution from an excited state, which in principle allows us to start the fit at earlier times. But for the sake of simplicity we will use the same time interval for both.

We can also compute the corresponding $\chi^2$ using:

$$ \chi^2=\sum_{t,t'}[\bar{E}(t)-f(t)](cov(t,t'))^{-1}[\bar{E}(t')-f(t')] $$

where $cov(t,t')$ is the covariance matrix

$$ cov(t,t')=\frac{N}{N-1}\frac{1}{N_b}\sum_{b=1}^{N_b}[E_b(t)-\bar{E}(t)][E_b(t')-\bar{E}(t')] $$

In order to obtain the parameters of the fit we have to minimize the $\chi^2$. A way to do so is creating a function for the $\chi^2$ and using $\texttt{minimize( )}$ with some initials guesses for the parameters.

In [ ]:
# Plot the effective mass with the fits


### Choosing the best fit

**You don't have to do this**

Ideally we would like to repeat the fits for several time intervals and find the best parameters. They would be the ones that minimize $\chi^2/\text{dof}$, where $\text{dof}$ are the degrees of freedom in each fit (sample size minus number of parameters fitted).

Then with this we would obtain the central values for the parameters $\bar{k}$ and $\bar{a}$, $\bar{b}$, $\bar{c}$ for each time range. To obtain their uncertainty we repeat the same steps but minimizing

$$ \chi_b^2=\sum_{t,t'}[E_b(t)-f(t)](cov(t,t'))^{-1}[E_b(t')-f(t')] $$

where the mean value of the effective energy $\bar{E}(t)$ is replaced by the effective mass of each bootstrap sample $b$, $E_b(t)$.

We obtain $N_b$ values for {k} and {a,b,c} that minimize this equation for each value of $b \in (1,Nb)$. In order to obtain the statistical uncertainty, we proceed to use

$$ \sigma_x=\frac{q_{5/6}(x_b-\bar{x})-q_{1/6}(x_b-\bar{x})}{2} $$

where $x_b$ are the values of $k$ (or $c$) which minimize each $\chi_b^2/\text{dof}$, $\bar{x}$ the value of $k$ (or $c$) that minimizes $\chi^2/\text{dof}$ and $q_n$ the $n$th quantile.

The best fit will be the one with smallest $\chi^2/\text{dof}$ from among all the fits obtained for different time ranges.

Finally, we assign a systematic uncertainty, $\sigma_\text{syst}$, calculated as the biggest difference between the central value $\bar{k}$ (or $\bar{c}$) of each fit and the central value of the best fit. Statistical and systematic uncertainties are
combined in quadrature to obtain the total uncertainty

$$\sigma_t = \sqrt{\sigma_\text{stat}^2 + \sigma_\text{syst}^2}$$

